In [ ]:
%%bash

pip install lightning torchinfo

In [ ]:
import numpy as np
import pandas as pd
from sklearn import model_selection, pipeline, preprocessing
import torch
from torch import nn, optim, utils
import torchinfo
import lightning as L

### Define some utility functions

The code in the cell below defines a few utility functions that will make our life easier.

In [ ]:
def clip_gradients_(
    clip_grad_strategy,
    model_fn,
    clip_value=None,
    error_if_nonfinite=False,
    max_norm=None,
    norm_type=2.0):
    if clip_grad_strategy == "value" and clip_value is not None:
        nn.utils.clip_grad_value_(
            model_fn.parameters(),
            clip_value
        )
    elif clip_grad_strategy == "norm" and max_norm is not None:
        nn.utils.clip_grad_norm_(
            model_fn.parameters(),
            max_norm,
            norm_type,
            error_if_nonfinite
        )
    elif clip_grad_strategy is None:
        pass
    else:
        raise NotImplementedError()


def compute_average_loss(dataloader, loss_fn, model_fn):
    total_loss = torch.zeros(1, 1)
    for features, targets in dataloader:
        predictions = model_fn(features)
        total_loss += loss_fn(predictions, targets)
    average_loss = total_loss / len(dataloader)
    return average_loss


def initialize_linear_layer(
    in_features,
    out_features,
    init_strategy_=nn.init.kaiming_uniform_):
    linear_layer = nn.Linear(in_features, out_features)
    init_strategy_(linear_layer.weight)
    return linear_layer


def make_mlp_classifier(
    input_size,
    hidden_sizes=None,
    output_size=2,
    activation_fn=None,
    init_strategy_=nn.init.kaiming_uniform_,
    batch_normalization=False):
    modules = []
    hidden_sizes = [] if hidden_sizes is None else hidden_sizes
    for hidden_size in hidden_sizes:
        hidden_layer = initialize_linear_layer(
            input_size,
            hidden_size,
            init_strategy_,
        )
        modules.append(hidden_layer)
        if batch_normalization:
            modules.append(nn.BatchNorm1d(hidden_size))
        if activation_fn is not None:
            modules.append(activation_fn)
        input_size=hidden_size
    output_layer = initialize_linear_layer(
            input_size,
            output_size,
            init_strategy_,
    )
    modules.append(output_layer)
    modules.append(nn.LogSoftmax(dim=1))
    model_fn = nn.Sequential(*modules)
    return model_fn, nn.NLLLoss()


In [ ]:
def fit(
    loss_fn,
    model_fn,
    optimizer,
    train_dataloader,
    val_dataloader,
    clip_grad_strategy=None,
    clip_value=None,
    error_if_nonfinite=False,
    log_epochs=1,
    max_epochs=1,
    max_norm=None,
    norm_type=2.0):

    history = {
        "epoch": [],
        "average_train_loss": [],
        "average_val_loss": []
    }

    for epoch in range(max_epochs):
        total_train_loss = torch.zeros(1, 1)
        model_fn = model_fn.train()
        for features, targets in train_dataloader:

            # forward pass
            predictions = model_fn(features)
            loss = loss_fn(predictions, targets)
            total_train_loss += loss

            # backward pass
            loss.backward()
            clip_gradients_(
                clip_grad_strategy,
                model_fn,
                clip_value,
                error_if_nonfinite,
                max_norm,
                norm_type
            )
            optimizer.step()
            optimizer.zero_grad()

        average_train_loss = total_train_loss / len(train_dataloader)
        history["epoch"].append(epoch)
        history["average_train_loss"].append(average_train_loss.item())

        # validation after every training epoch
        model_fn = model_fn.eval()
        with torch.inference_mode():
            average_val_loss = compute_average_loss(
                val_dataloader,
                loss_fn,
                model_fn
            )
        history["average_val_loss"].append(average_val_loss.item())


        if epoch % log_epochs == 0:
            message = f"Epoch {epoch}, Average train Loss {average_train_loss.item():.4f}, Average val Loss {average_val_loss.item():.4f}"
            print(message)

    history_df = (pd.DataFrame.from_dict(history)
                              .set_index("epoch"))
    return history_df


# Transfer Learning

In this section we will train a DNN model on the MNIST dataset and then use this pre-trained model as a starting point for training a model to classify an Arabic version of the MNIST dataset.

## Load the MNIST data

In [ ]:
INPUT_SIZE = 784
OUTPUT_SIZE = 10

_train_data = pd.read_csv(
    "./sample_data/mnist_train_small.csv",
    header=None,
    names=["label"] + [f"p{i}" for i in range(INPUT_SIZE)],
)
mnist_train_data, mnist_val_data = model_selection.train_test_split(
    _train_data,
    test_size=0.1,
    stratify=_train_data.loc[:, "label"]
)

mnist_test_data = pd.read_csv(
    "./sample_data/mnist_test.csv",
    header=None,
    names=["label"] + [f"p{i}" for i in range(INPUT_SIZE)],
)

### Create preprocessing pipelines

In [ ]:
features_preprocessor = pipeline.make_pipeline(
    preprocessing.MinMaxScaler(),
    preprocessing.FunctionTransformer(lambda arr: arr.astype(np.float32)),
    preprocessing.FunctionTransformer(lambda arr: torch.from_numpy(arr))
)

target_preprocessor = pipeline.make_pipeline(
    preprocessing.FunctionTransformer(lambda df: df.to_numpy()),
    preprocessing.FunctionTransformer(lambda arr: torch.from_numpy(arr))
)


### Create Datasets and DataLoaders

In [ ]:
BATCH_SIZE = 64
NUM_WORKERS = 2

# create the training dataset and dataloader
_train_features_tensor = features_preprocessor.fit_transform(
    mnist_train_data.drop("label", axis=1)
)

_train_target_tensor = target_preprocessor.fit_transform(
    mnist_train_data.loc[:, "label"]
)

_train_dataset = utils.data.TensorDataset(
    _train_features_tensor,
    _train_target_tensor
)

mnist_train_dataloader = utils.data.DataLoader(
    _train_dataset,
    batch_size=BATCH_SIZE,
    num_workers=NUM_WORKERS,
)

# create the validation dataset and dataloader
_val_features_tensor = features_preprocessor.transform(
    mnist_val_data.drop("label", axis=1)
)

_val_target_tensor = target_preprocessor.transform(
    mnist_val_data.loc[:, "label"]
)

_val_dataset = utils.data.TensorDataset(
    _val_features_tensor,
    _val_target_tensor
)

mnist_val_dataloader = utils.data.DataLoader(
    _val_dataset,
    batch_size=BATCH_SIZE,
    num_workers=NUM_WORKERS,
)

# create the test dataset and dataloader
_test_features_tensor = features_preprocessor.transform(
    mnist_test_data.drop("label", axis=1)
)

_test_target_tensor = target_preprocessor.transform(
    mnist_test_data.loc[:, "label"]
)

_test_dataset = utils.data.TensorDataset(
    _test_features_tensor,
    _test_target_tensor
)

mnist_test_dataloader = utils.data.DataLoader(
    _test_dataset,
    batch_size=BATCH_SIZE,
    num_workers=NUM_WORKERS,
)


## Train a DNN on the MNIST data

In [ ]:
HIDDEN_SIZE = 100
LEARNING_RATE = 1e-2
MAX_EPOCHS = 20

pretrained_mnist_model_fn, loss_fn = make_mlp_classifier(
    input_size=INPUT_SIZE,
    hidden_sizes=[HIDDEN_SIZE, HIDDEN_SIZE, HIDDEN_SIZE],
    output_size=OUTPUT_SIZE,
    activation_fn=nn.SELU(),
    init_strategy_=nn.init.xavier_normal_
)

optimizer = optim.SGD(
    pretrained_mnist_model_fn.parameters(),
    lr=LEARNING_RATE
)

history = fit(
    loss_fn,
    pretrained_mnist_model_fn,
    optimizer,
    mnist_train_dataloader,
    mnist_val_dataloader,
    max_epochs=MAX_EPOCHS
)

## Download Arabic Handwritten Digits Data

Here we will download only the test dataset on 10k images. All images have the same size 28x28 = 784 pixels as the orginal MNIST data; there are also the same number of classes in this dataset.

In [ ]:
%%bash

gdown 1_aWjSUmpBlLSTSJF4xhmdZnzMoK4AMRO
gdown 1Syln7zCy9Ue8x_F5qgq_0EHvaL_h_oL5
gdown 1QQUsu7QciAZ6KXt6u-ZaMT_61iNjgy0a

In [ ]:
_test_features = pd.read_csv(
    "/content/csvTestImages 10k x 784.csv",
    header=None,
    names=[f"p{i}" for i in range(INPUT_SIZE)],
)

_test_target = pd.read_csv(
    "/content/csvTestLabel 10k x 1.csv",
    header=None,
    names=["label"],
)
_splits = model_selection.train_test_split(
    _test_features,
    _test_target,
    test_size=0.1,
    shuffle=True,
    stratify=_test_target.loc[:, "label"]
)

arabic_mnist_train_features, arabic_mnist_val_features = _splits[:2]
arabic_mnist_train_target, arabic_mnist_val_target = _splits[2:]

In [ ]:
# create the training dataset and dataloader
_train_features_tensor = features_preprocessor.fit_transform(
    arabic_mnist_train_features
)

_train_target_tensor = target_preprocessor.fit_transform(
    arabic_mnist_train_target.loc[:, "label"]
)

_train_dataset = utils.data.TensorDataset(
    _train_features_tensor,
    _train_target_tensor
)

arabic_mnist_train_dataloader = utils.data.DataLoader(
    _train_dataset,
    batch_size=BATCH_SIZE,
    num_workers=NUM_WORKERS,
)

# create the validation dataset and dataloader
_val_features_tensor = features_preprocessor.transform(
    arabic_mnist_val_features)

_val_target_tensor = target_preprocessor.transform(
    arabic_mnist_val_target.loc[:, "label"]
)

_val_dataset = utils.data.TensorDataset(
    _val_features_tensor,
    _val_target_tensor
)

arabic_mnist_val_dataloader = utils.data.DataLoader(
    _val_dataset,
    batch_size=BATCH_SIZE,
    num_workers=NUM_WORKERS,
)

### Logistic Regression Benchmark

First, we can use logistic regression as a benchmark model. Our more elaborate models should seek to outperform this benchmark.

In [ ]:
benchmark_model_fn, loss_fn = make_mlp_classifier(
    input_size=INPUT_SIZE,
    output_size=OUTPUT_SIZE,
)

optimizer = optim.SGD(
    benchmark_model_fn.parameters(),
    lr=LEARNING_RATE
)

history = fit(
    loss_fn,
    benchmark_model_fn,
    optimizer,
    arabic_mnist_train_dataloader,
    arabic_mnist_val_dataloader,
    max_epochs=MAX_EPOCHS
)

### Freeze the pre-trained backbone and fine-tune the classifier head

In [ ]:
torchinfo.summary(pretrained_mnist_model_fn)

In [ ]:
import copy


arabic_mnist_model_fn = copy.deepcopy(pretrained_mnist_model_fn)

# freeze layers 0 through 5 (inclusive!)
for i in range(0, 5 + 1):
    for p in arabic_mnist_model_fn[i].parameters():
        p.requires_grad = False

In [ ]:
torchinfo.summary(arabic_mnist_model_fn)

In [ ]:
optimizer = optim.SGD(
    arabic_mnist_model_fn.parameters(),
    lr=LEARNING_RATE
)

history = fit(
    loss_fn,
    arabic_mnist_model_fn,
    optimizer,
    arabic_mnist_train_dataloader,
    arabic_mnist_val_dataloader,
    max_epochs=MAX_EPOCHS
)

Just fine-tuning the classifier head didn't seem to work very well. Lets try to un-freeze additional layers.

In [ ]:
arabic_mnist_model_fn = copy.deepcopy(pretrained_mnist_model_fn)

# freeze layers 0 through 3 (inclusive!)
for i in range(0, 3 + 1):
    for p in arabic_mnist_model_fn[i].parameters():
        p.requires_grad = False

In [ ]:
torchinfo.summary(arabic_mnist_model_fn)

In [ ]:
optimizer = optim.SGD(
    arabic_mnist_model_fn.parameters(),
    lr=LEARNING_RATE
)

history = fit(
    loss_fn,
    arabic_mnist_model_fn,
    optimizer,
    arabic_mnist_train_dataloader,
    arabic_mnist_val_dataloader,
    max_epochs=MAX_EPOCHS
)

Now we are seeing some improvement!

### Fine-tune the MNIST DNN

You can also try unfreezing all of the layers and fine-tune the pre-trained model on your new data.

In [ ]:
arabic_mnist_model_fn = copy.deepcopy(pretrained_mnist_model_fn)

In [ ]:
optimizer = optim.SGD(
    arabic_mnist_model_fn.parameters(),
    lr=LEARNING_RATE
)

history = fit(
    loss_fn,
    arabic_mnist_model_fn,
    optimizer,
    arabic_mnist_train_dataloader,
    arabic_mnist_val_dataloader,
    max_epochs=MAX_EPOCHS
)

### Exercise:

Compare the performance of the original MNIST model at predicting the Arabic Handwritten digits with the performance of the Arabic MNIST model that was fine-tuned on the Arabic MNIST data.

# Unsupervised pre-training

Often you will have a large amount of unlabeled data and a small amount of labeled data. In this situation, one possible solution is to use unsupervised pre-training.

In [ ]:
arabic_mnist_unlabeled_features = pd.read_csv(
    "/content/csvTrainImages 60k x 784.csv",
    header=None,
    names=[f"p{i}" for i in range(INPUT_SIZE)],
)


## Generate an embedding

When applying unsupervised pre-training you first need to find a "good" embedding of your unlabeled features. For computer vision applications you would want to use more powerful models such as autoencoders, generative adversarial models (GANs) or similar to generate your embeddings. Here we just use PCA.

In [ ]:
from sklearn import decomposition

In [ ]:
encoder_pipeline = pipeline.make_pipeline(
    decomposition.PCA(n_components=0.95),
)

In [ ]:
feature_embedding = encoder_pipeline.fit_transform(arabic_mnist_unlabeled_features)

In [ ]:
_, input_size = feature_embedding.shape

How do we define a "good" embedding? Reconstruction error! Once we have a feature embedding, we can invert the embedding transformation in order to reconstruct the original features. Once we have a reconstruction of the original features we can estimate the reconstruction error by comparing the original features and the reconstructed features.

A good embedding will have a small reconstruction error.

In [ ]:
from sklearn import metrics

In [ ]:
# here we estimate the reconstruction error using mean squared error
reconstructed_features = encoder_pipeline.inverse_transform(feature_embedding)
metrics.mean_squared_error(
    arabic_mnist_unlabeled_features,
    reconstructed_features
)

### Exercise:

How could we improve the performance of our feature encoder pipeline? What impact will improving the performance of the feature encoder pipeline have on the performance of the DNN trained on the encoded features?

## Use encoder pipeline to embed your labeled data

Hopefully our "good" embedding of the unlabeled features has learned useful information for our supervised classification task. Now we use the trained encoder pipeline to embed our labeled features.

In [ ]:
embedded_train_features = encoder_pipeline.transform(arabic_mnist_train_features)
embedded_val_features = encoder_pipeline.transform(arabic_mnist_val_features)


Now we need to prepare out data for our DNN.

In [ ]:
features_preprocessor = pipeline.make_pipeline(
    preprocessing.StandardScaler(),
    preprocessing.FunctionTransformer(lambda arr: arr.astype(np.float32)),
    preprocessing.FunctionTransformer(lambda arr: torch.from_numpy(arr))
)

target_preprocessor = pipeline.make_pipeline(
    preprocessing.FunctionTransformer(lambda df: df.to_numpy()),
    preprocessing.FunctionTransformer(lambda arr: torch.from_numpy(arr))
)

# create the training dataset and dataloader
_train_features_tensor = features_preprocessor.fit_transform(
    embedded_train_features
)

_train_target_tensor = target_preprocessor.fit_transform(
    arabic_mnist_train_target.loc[:, "label"]
)

_train_dataset = utils.data.TensorDataset(
    _train_features_tensor,
    _train_target_tensor
)

embedded_train_dataloader = utils.data.DataLoader(
    _train_dataset,
    batch_size=BATCH_SIZE,
    num_workers=NUM_WORKERS,
)

# create the validation dataset and dataloader
_val_features_tensor = features_preprocessor.transform(
    embedded_val_features)

_val_target_tensor = target_preprocessor.transform(
    arabic_mnist_val_target.loc[:, "label"]
)

_val_dataset = utils.data.TensorDataset(
    _val_features_tensor,
    _val_target_tensor
)

embedded_val_dataloader = utils.data.DataLoader(
    _val_dataset,
    batch_size=BATCH_SIZE,
    num_workers=NUM_WORKERS,
)

### Train a DNN using the embedded features

In [ ]:
unsupervised_pretraining_model_fn, loss_fn = make_mlp_classifier(
    input_size=input_size,
    hidden_sizes=[HIDDEN_SIZE, HIDDEN_SIZE, HIDDEN_SIZE],
    output_size=OUTPUT_SIZE,
    activation_fn=nn.SELU(),
    init_strategy_=nn.init.xavier_normal_
)

optimizer = optim.SGD(
    unsupervised_pretraining_model_fn.parameters(),
    lr=LEARNING_RATE
)

history = fit(
    loss_fn,
    unsupervised_pretraining_model_fn,
    optimizer,
    embedded_train_dataloader,
    embedded_val_dataloader,
    max_epochs=MAX_EPOCHS
)

### Exercise:

Compare the performance of the transfer learning and unsupervised pre-training approaches to classifying the Arabic Handwritten Digits images.